<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>

# Plot variables relevant to soil biogeochemistry from ELM output.

#### ----------------------------------------------------------------------------------------------------------------------

<br>

To run this script and generate the resulting ELM plots we need specific Python libraries.  This step loads those required libraries

In [ ]:
import matplotlib.pyplot as plt
import xarray
import os
import ipywidgets as widgets
import glob, numpy

Next, we locate the output data and identify which sites there is output for. This code creates a dropdown menu to pick the model run you want to plot.

In [ ]:
output_rootdir=os.path.expanduser('~')+'/output/cime_run_dirs/'
cases=sorted(glob.glob("%s*20TR*" % output_rootdir))
cases=[os.path.basename(x) for x in cases]
cases_dropdown = widgets.Dropdown(options=cases,
                                description='Choose Case Name:',
                                style={'description_width':'auto'},
                                layout={'width':'max-content'},
                                disabled=False)

display(cases_dropdown)

Here, we are using the combined output file over all years of the run. We read in the list of variables, units, and longer descriptions from the dataset. We use the time variable from the dataset to determine the start and end years.

In [ ]:
output_file=output_rootdir + cases_dropdown.value + '/run/ELM_output.nc'

# Read in the list of variable, units, and longer descriptions from the dataset
with xarray.open_dataset(output_file) as data:
    # Determine start and end year of the dataset
    data_startyear=data.time[0].item().year
    data_endyear=data.time[-1].item().year

startyear_picker = widgets.BoundedIntText(value=data_startyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose start year',style={'description_width':'auto'})
endyear_picker = widgets.BoundedIntText(value=data_endyear, min=data_startyear, max=data_endyear, step=1,
                                          description='Choose end year',style={'description_width':'auto'})

smoothing_picker = widgets.IntText(value=0,description='Choose smoothing on fluxes (months)',style={'description_width':'auto'})

display(startyear_picker)
display(endyear_picker)
display(smoothing_picker)

Using the fields above, you can pick the start and end year of the time series to visualize.

Next, we read in the dataset and select the time portion based on start and end year picked above.

In [ ]:
year_start=startyear_picker.value
year_end=endyear_picker.value
smoothing=smoothing_picker.value

elm_output=xarray.open_dataset(output_file).squeeze().sel(time=slice(str(year_start),str(year_end)))

Now we set up a figure and plot the relevant variables in different panels.

In [ ]:
# Plot the carbon and nitrogen budgets
# Set up a figure with three axes
fig,a=plt.subplots(nrows=2,ncols=2,clear=True,num='Carbon budgets',figsize=(15,13))

ax=a[0,0]
elm_output['TOTSOMC'].plot(ax=ax,linestyle='-',color='black',label='Total SOM C')
elm_output['TOTLITC'].plot(ax=ax,linestyle='-',color='red',label='Total litter C')

ax.legend()
ax.set(title='Soil carbon pools',xlabel='Year',ylabel='Carbon stock (g C m$^{-2}$)')


# Soil N
ax=a[0,1]
elm_output['TOTSOMN'].plot(ax=ax,linestyle='-',color='black',label='Total SOM N')
elm_output['TOTLITN'].plot(ax=ax,linestyle='-',color='red',label='Total litter N')


# C fluxes
ax=a[1,0]
# Do smoothing if wanted
if smoothing>0:
    HR=elm_output['HR'].resample(time=str(smoothing)+'M').mean()
else:
    HR=elm_output['HR']

(HR*24*3600).plot(ax=ax,linestyle='-',color='brown',label='Ecosystem respiration')
ax.legend()
ax.set(title='C fluxes',xlabel='Year',ylabel='C flux (g C m$^{-2}$ day$^{-1}$)')


# N fluxes
ax=a[1,1]
# Do smoothing if wanted
if smoothing>0:
    NET_NMIN=elm_output['NET_NMIN'].resample(time=str(smoothing)+'M').mean()
    GROSS_NMIN=elm_output['GROSS_NMIN'].resample(time=str(smoothing)+'M').mean()
    SMINN_TO_PLANT=elm_output['SMINN_TO_PLANT'].resample(time=str(smoothing)+'M').mean()
    NDEP_TO_SMINN=elm_output['NDEP_TO_SMINN'].resample(time=str(smoothing)+'M').mean()
    NFIX_TO_SMINN=elm_output['NFIX_TO_SMINN'].resample(time=str(smoothing)+'M').mean()
else:
    NET_NMIN=elm_output['NET_NMIN']
    GROSS_NMIN=elm_output['GROSS_NMIN']
    SMINN_TO_PLANT=elm_output['SMINN_TO_PLANT']
    NDEP_TO_SMINN=elm_output['NDEP_TO_SMINN']
    NFIX_TO_SMINN=elm_output['NFIX_TO_SMINN']

(NET_NMIN*24*3600).plot(ax=ax,linestyle='-',color='blue',label='Net Nmin')
(GROSS_NMIN*24*3600).plot(ax=ax,linestyle='--',color='blue',label='Gross Nmin')
(SMINN_TO_PLANT*24*3600).plot(ax=ax,linestyle='-',color='green',label='Plant N uptake')

(NDEP_TO_SMINN*24*3600).plot(ax=ax,linestyle='-',color='C0',label='Atmospheric N dep')
(NFIX_TO_SMINN*24*3600).plot(ax=ax,linestyle='-',color='C1',label='N fixation')
ax.legend()
ax.set(title='N fluxes',xlabel='Year',ylabel='N flux (g N m$^{-2}$ day$^{-1}$)')


Next plot depth profiles of soil carbon and C:N ratios

In [ ]:
f,a=plt.subplots(nrows=2,ncols=2,gridspec_kw={'width_ratios':[1,0.5]},figsize=(6,8))

# Soil C/N depth distribution
soilC=elm_output['LITR1C_vr']+elm_output['LITR2C_vr']+elm_output['LITR3C_vr']+elm_output['SOIL1C_vr']+elm_output['SOIL2C_vr']+elm_output['SOIL3C_vr']+elm_output['SOIL4C_vr']
soilN=elm_output['LITR1N_vr']+elm_output['LITR2N_vr']+elm_output['LITR3N_vr']+elm_output['SOIL1N_vr']+elm_output['SOIL2N_vr']+elm_output['SOIL3N_vr']+elm_output['SOIL4N_vr']

ax=a[0,0]
soilC.T.plot(ax=ax)
maxdepth=2.0
ax.set_ylim(maxdepth,0.0)
ax.set_ylabel('Soil depth (m)')
ax.set_title('Total soil carbon by depth')
ax.set_xlabel('Year')

ax=a[0,1]
ax.plot(soilC.mean(dim='time'),soilC['levdcmp'])
ax.set_ylim(maxdepth,0.0)
ax.set_ylabel('Soil depth (m)')
ax.set_title('Total soil carbon by depth')
ax.set_xlabel('Soil carbon density (g C m$^{-3}$)')

ax=a[1,0]
(soilC/soilN).T.plot(ax=ax)
ax.set_ylim(maxdepth,0.0)
ax.set_ylabel('Soil depth (m)')
ax.set_title('Soil C:N ratio by depth')
ax.set_xlabel('Year')

ax=a[1,1]
ax.plot((soilC/soilN).mean(dim='time'),soilC['levdcmp'])
ax.set_ylim(maxdepth,0.0)
ax.set_ylabel('Soil depth (m)')
ax.set_title('Soil C:N ratio by depth')
ax.set_xlabel('Soil C:N ratio')

<center><img src="https://github.com/FASSt-simulation/fasst_simulation_tools/raw/main/images/site-logo.png" width=150 height=150 alt="NGEE-Arctic Logo"/></center>